In [1]:
import json
from preprocessor.preprocessor_pytorch import Preprocessor
from model.model_pytorch import Model
from postprocessor.postprocessor import Postprocesser
from evaluator.evaluator_pytorch import Evaluator
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import os
import time
import pickle

class ReversePrediction():
    def set_seed(self, seed_value):
        np.random.seed(seed_value)
        tf.random.set_seed(seed_value)
        random.seed(seed_value)
        torch.manual_seed(seed_value)


    def run(self, params): 
        self.set_seed(42)
        preprocessor = Preprocessor(params)
        X_train, y_train, X_val, y_val, X_test, y_test, test_dates, X_newest, x_newest_date, y_newest_date, target_symbol_data = preprocessor.get_multiple_data()
        
        start_time = time.time()
        model_wrapper = Model(params=params)
        model, history, y_preds, online_training_losses, online_training_acc = \
            model_wrapper.run(X_train, y_train, X_test, y_test, X_val, y_val)
        end_time = time.time()
        execution_time = end_time - start_time
        
        y_preds = torch.tensor(y_preds, dtype=torch.float32)
        y_preds = preprocessor.change_values_after_first_reverse_point(y_preds)

        postprocessor = Postprocesser()
        X_train, X_test, X_newest, y_train, y_test, y_preds = \
            postprocessor.check_shape(X_train, X_test, X_newest, y_train, y_test, y_preds, reshape=params['model_params'][params['model_type']]['reshape'])
        test_trade_signals = postprocessor.process_signals(y_test, test_dates, False)
        pred_trade_signals = postprocessor.process_signals(y_preds, test_dates, params['filter'])
        evaluator = Evaluator(params=params)
        model_summary, pred_days_difference_results, backtesting_report, trade_summary, execution_time, confusion_matrix_info, y_test, y_preds = evaluator.generate_numericale_data(model,
                        y_test, y_preds, target_symbol_data, pred_trade_signals, test_trade_signals, execution_time)
        
        return model_summary, pred_days_difference_results, backtesting_report, trade_summary, execution_time, confusion_matrix_info, y_test, y_preds, test_trade_signals, pred_trade_signals


In [2]:
# with open('parameters.json', 'r') as f:
#     params = json.load(f)

In [3]:
params = {
  "train_indices": ["^GSPC", "^IXIC", "^DJI", "^RUA"],
  "test_indices": "^GSPC",
  "start_date": "2003-10-23",
  "stop_date": "2023-10-23",
  "features_params": [
      {
          "type": "Trend",
          "method": "MA",
          "oder_days": 20,
          "ma_days": 20,
          "trend_days": 5
      },
      {
          "type": "MACD",
          "fastperiod": 5,
          "slowperiod": 10,
          "signalperiod": 9
      },
      {
          "type": "ROC",
          "trend_days": 5
      },
      {
          "type": "Stochastic Oscillator",
          "trend_days": 5
      },
      {
          "type": "CCI",
          "timeperiod": 14
      },
      {
          "type": "RSI",
          "timeperiod": 14
      },
      {
          "type": "VMA",
          "timeperiod": 20
      },
      {
          "type": "pctChange"
      },
      {
          "type": "13W Treasury Yield",
          "start_date": "2001-01-01",
          "end_date": "2021-01-01"
      },
      {
          "type": "5Y Treasury Yield",
          "start_date": "2001-01-01",
          "end_date": "2021-01-01"
      },
      {
          "type": "10Y Treasury Yield",
          "start_date": "2001-01-01",
          "end_date": "2021-01-01"
      },
      {
          "type": "30Y Treasury Yield",
          "start_date": "2001-01-01",
          "end_date": "2021-01-01"
      },
      {
          "type": "Bollinger Bands",
          "timeperiod": 20,
          "nbdevup": 2,
          "nbdevdn": 2
      },
      {
          "type": "ATR",
          "timeperiod": 14
      },
      {
          "type": "OBV"
      },
      {
          "type": "Parabolic SAR",
          "start": 0.02,
          "increment": 0.02,
          "maximum": 0.2
      },
      {
          "type": "MOM",
          "timeperiod": 10
      },
      {
          "type": "Williams %R",
          "lookback_period": 14
      },
      {
          "type": "Chaikin MF",
          "timeperiod": 20
      }
  ],
  "data_cleaning": {
      "clean_type": "MissingData",
      "strategy": "auto"
  },
  "split_ratio": 0.7,
  "target_col": "Trend",
  "feature_cols": [
      "Open",
      "High",
      "Low",
      "Close",
      "Adj Close",
      "Volume",
      "Trend",
      "MACD_dif",
      "MACD_dem",
      "MACD_histogram",
      "ROC",
      "StoK",
      "StoD",
      "CCI",
      "RSI",
      "VMA",
      "pctChange",
      "13W Treasury Yield",
      "5Y Treasury Yield",
      "10Y Treasury Yield",
      "30Y Treasury Yield",
      "upperband",
      "middleband",
      "lowerband",
      "ATR",
      "OBV",
      "Parabolic SAR",
      "MOM",
      "Williams %R",
      "Chaikin MF"
  ],
  "look_back": 32,
  "predict_steps": 8,
  "train_slide_steps": 32,
  "test_slide_steps": 8,
  "model_params": {
      "LeNet": {
          "conv_1_in_channels": 19,
          "conv_1_out_channels": 8,
          "conv_1_kernel": 3,
          "conv_1_padding": 1,
          "MaxPool2d_1_kernel_size": 2,
          "fc_1_out_features": 512,
          "reshape": "False"
      },
      "LeNet_2": {
          "conv_1_in_channels": 19,
          "conv_1_out_channels": 6,
          "conv_1_kernel": 3,
          "conv_1_padding": 1,
          "MaxPool2d_1_kernel_size": 2,
          "conv_2_out_channels": 8,
          "conv_2_kernel": 3,
          "conv_2_padding": 1,
          "MaxPool2d_2_kernel_size": 2,
          "fc_1_out_features": 512,
          "reshape": "False"
      },
      "RNN": {
          "hidden_size": 32,
          "num_layers": 1,
          "dropout": 0.2,
          "reshape": "False"
      },
      "LSTM": {
          "hidden_size": 32,
          "num_layers": 1,
          "dropout": 0.2,
          "reshape": "False"
      },
      "CNN_LSTM": {
          "conv_1_in_channels": 19,
          "conv_1_out_channels": 6,
          "conv_1_kernel": 3,
          "conv_1_padding": 1,
          "MaxPool2d_1_kernel_size": 2,
          "conv_2_out_channels": 8,
          "conv_2_kernel": 3,
          "conv_2_padding": 1,
          "MaxPool2d_2_kernel_size": 2,
          "fc_1_out_features": 512,
          "hidden_size": 32,
          "num_layers": 1,
          "dropout": 0.2,
          "reshape": "True"
      },
      "DNN_5layers": {
          "reshape": "False"
      },
      "DummyClassifier": {
          "reshape": "False"
      }
  },
  "model_type": "DNN_5layers",
  "training_epoch_num": 1000,
  "online_training_epoch_num": 10,
  "learning_rate": 0.001,
  "online_train_learning_rate": 0.001,
  "patience": 10,
  "min_delta": 0.01,
  "apply_weight": "True",
  "data_update_mode": "append",
  "filter": "False",
  "trade_strategy": "all",
  "filter_reverse_trend": "True",
  "trained_model_path": "models/trained_model.pt",
  "online_trained_model_path": "models/online_trained_model.pt",
  "plot_training_curve_save_path": "plots/plot_training_curve.png",
  "online_training_curve_save_path": "plots/online_training_curve.png",
  "confusion_matrix_save_path": "plots/confusion_matrix.png",
  "predictions_save_path": "plots/predictions.png",
  "filtered_predictions_save_path": "plots/filtered_predictions.png",
  "pred_trade_signals_plot_save_path": "plots/pred_trade_signals_plot.png",
  "test_trade_signals_plot_save_path": "plots/test_trade_signals_plot.png",
  "pred_days_difference_bar_chart_save_path": "plots/pred_days_difference_bar_chart.png",
  "roc_pr_curve_save_path": "plots/roc_pr_curve.png",
  "report_save_path": "reports/report.html",
  "summary_save_path": "reports/summary.json"
}

In [4]:
rp = ReversePrediction() 
rp.set_seed(42)

preprocessor = Preprocessor(params)
X_train, y_train, X_val, y_val, X_test, y_test, test_dates, X_newest, x_newest_date, y_newest_date, target_symbol_data = preprocessor.get_multiple_data()


[*********************100%***********************]  1 of 1 completed


c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 0 # up trend
c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 1 # down trend


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 0 # up trend
c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 1 # down trend


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 0 # up trend
c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 1 # down trend


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 0 # up trend
c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 1 # down trend


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Training set shape: torch.Size([345, 32, 30])
Validation set shape: torch.Size([87, 32, 30])


In [5]:
start_time = time.time()
model_wrapper = Model(params=params)
model, history, y_preds, online_training_losses, online_training_acc = \
    model_wrapper.run(X_train, y_train, X_test, y_test, X_val, y_val)
end_time = time.time()
execution_time = end_time - start_time


 12%|█▏        | 117/1000 [00:01<00:08, 101.90it/s]


In [6]:
y_preds = torch.tensor(y_preds, dtype=torch.float32)
y_preds = preprocessor.change_values_after_first_reverse_point(y_preds)


In [7]:
y_pred_newest = model.forward(X_newest)
y_pred_newest = torch.tensor(y_pred_newest, dtype=torch.float32)
y_pred_newest = preprocessor.change_values_after_first_reverse_point(y_pred_newest)

C:\Users\e4903\AppData\Local\Temp\ipykernel_29584\2634262193.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred_newest = torch.tensor(y_pred_newest, dtype=torch.float32)


In [8]:
postprocessor = Postprocesser()
X_train, X_test, X_newest, y_train, y_test, y_preds = \
    postprocessor.check_shape(X_train, X_test, X_newest, y_train, y_test, y_preds, reshape=params['model_params'][params['model_type']]['reshape'])
test_trade_signals = postprocessor.process_signals(y_test, test_dates, False)
pred_trade_signals = postprocessor.process_signals(y_preds, test_dates, params['filter'])
newest_trade_signals = postprocessor.process_signals(y_pred_newest, y_newest_date, False)

In [9]:
newest_trade_signals

,Date,Signal
0,2023-10-23,
1,2023-10-24,
2,2023-10-25,
3,2023-10-26,
4,2023-10-27,
5,2023-10-30,
6,2023-10-31,
7,2023-11-01,


In [10]:
evaluator = Evaluator(params=params)
model_summary, pred_days_difference_results, backtesting_report, trade_summary, execution_time, confusion_matrix_info, y_test, y_preds = evaluator.generate_numericale_data(model,
                y_test, y_preds, target_symbol_data, pred_trade_signals, test_trade_signals, execution_time)



In [11]:
# with open('parameters.json', 'r') as f:
#     params = json.load(f)
    
rp = ReversePrediction()
model_summary, pred_days_difference_results, backtesting_report, trade_summary, execution_time, confusion_matrix_info, y_test, y_preds, test_trade_signals, pred_trade_signals = rp.run(
    params)
confusion_matrix_info_json = json.dumps(confusion_matrix_info)
model_summary_json = json.dumps(model_summary)
backtesting_report_json = json.dumps(backtesting_report)
pred_days_difference_results_json = pred_days_difference_results.to_json()
trade_summary_json = json.dumps(trade_summary)
execution_time_json = json.dumps(execution_time)
newest_trade_signals_json = newest_trade_signals.to_json()

response = {
    'msg': 'Received!',
    'usingData': params,
    'confusion_matrix_info': confusion_matrix_info_json,
    'model_summary': model_summary_json,
    'backtesting_report': backtesting_report_json,
    'pred_days_difference_results': pred_days_difference_results_json,
    'trade_summary': trade_summary_json,
    'execution_time': execution_time_json,
    'newest_trade_signals': newest_trade_signals_json
}


c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 0 # up trend
c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 1 # down trend
c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\0321\preprocessor\preprocessor_pytorch.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

In [12]:
response

{'msg': 'Received!',
 'usingData': {'train_indices': ['^GSPC', '^IXIC', '^DJI', '^RUA'],
  'test_indices': '^GSPC',
  'start_date': '2003-10-23',
  'stop_date': '2023-10-23',
  'features_params': [{'type': 'Trend',
    'method': 'MA',
    'oder_days': 20,
    'ma_days': 20,
    'trend_days': 5},
   {'type': 'MACD', 'fastperiod': 5, 'slowperiod': 10, 'signalperiod': 9},
   {'type': 'ROC', 'trend_days': 5},
   {'type': 'Stochastic Oscillator', 'trend_days': 5},
   {'type': 'CCI', 'timeperiod': 14},
   {'type': 'RSI', 'timeperiod': 14},
   {'type': 'VMA', 'timeperiod': 20},
   {'type': 'pctChange'},
   {'type': '13W Treasury Yield',
    'start_date': '2001-01-01',
    'end_date': '2021-01-01'},
   {'type': '5Y Treasury Yield',
    'start_date': '2001-01-01',
    'end_date': '2021-01-01'},
   {'type': '10Y Treasury Yield',
    'start_date': '2001-01-01',
    'end_date': '2021-01-01'},
   {'type': '30Y Treasury Yield',
    'start_date': '2001-01-01',
    'end_date': '2021-01-01'},
   {'type